In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import pairwise_distances_argmin_min

In [2]:
df = pd.read_excel("ihdp_data.xlsx")
df = df.drop(['y_cfactual','mu0','mu1'], axis = 1)
df.rename(columns={'y_factual': 'outcome'}, inplace=True)
df.head()

,treatment,outcome,x1,x2,x3,x4,x5,x6,x7,x8,...,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25
0,True,5.599916,-0.528603,-0.343455,1.128554,0.161703,-0.316603,1.295216,1,0,...,1,1,1,1,0,0,0,0,0,0
1,False,6.875856,-1.736945,-1.802002,0.383828,2.244320,-0.629189,1.295216,0,0,...,1,1,1,1,0,0,0,0,0,0
2,False,2.996273,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0,0,...,1,0,1,1,0,0,0,0,0,0
3,False,1.366206,0.390083,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0,0,...,1,0,1,1,0,0,0,0,0,0
4,False,1.963538,-1.045229,-0.602710,0.011465,0.161703,0.683672,-0.360940,1,0,...,1,1,1,1,0,0,0,0,0,0


In [3]:
df['treatment'].value_counts()

treatment
False    608
True     139
Name: count, dtype: int64

Propensity Score Matching

In [4]:
# Estimate propensity scores
np.random.seed(42)
model = LogisticRegression(random_state=42)
model.fit(df.drop(['treatment', 'outcome'], axis=1), df['treatment'])

LogisticRegression(random_state=42)

In [5]:
# Calculate the propensity scores 
df['propensity_score'] = model.predict_proba(df.drop(['treatment', 'outcome'], axis=1))[:,1]

One-to-one Matching

In [6]:
# Separate the treated and untreated subjects 
treated = df[df['treatment'] == 1]
untreated = df[df['treatment'] == 0]

In [7]:
# Find the closest untreated subjects to the treated subjects based on propensity scores
indices, _ = pairwise_distances_argmin_min(treated[['propensity_score']], untreated[['propensity_score']])

In [8]:
# Create a matched dataset where each treated subject is paired with the closest untreated subject
matched = treated.copy()
matched['matched_outcome'] = untreated.iloc[indices]['outcome'].values

In [9]:
# Calculate the Average Treatment Effect on the Treated (ATT) using the matched pairs
att_psm = (matched['outcome'] - matched['matched_outcome']).mean()
print("ATT (Training Data):", att_psm)

ATT (Training Data): 3.7768352642503777


Many-to-one Matching

In [10]:
from sklearn.neighbors import NearestNeighbors

def many_to_one_matching(treated, untreated, n_neighbors=5):
    # Initialize the NearestNeighbors object with the number of neighbors to find
    neighbors = NearestNeighbors(n_neighbors=n_neighbors)
    # Fit the NearestNeighbors model on the propensity scores of untreated subjects
    neighbors.fit(untreated[['propensity_score']])
    # Find the nearest neighbors for each treated subject based on propensity scores
    distances, indices = neighbors.kneighbors(treated[['propensity_score']])
    # Create a list of matched pairs (treated_index, untreated_index)
    # Each treated subject is paired with 'n_neighbors' nearest untreated subjects
    return [(treated.index[i], untreated.index[j]) for i in range(len(treated)) for j in indices[i]]

# Uses the kneighbors method to find the nearest neighbors for each treated subject
matched_indices = many_to_one_matching(treated, untreated)
# Creates a list of matched pairs
matched = pd.DataFrame([(i, j, treated.at[i, 'outcome'], untreated.at[j, 'outcome']) for i, j in matched_indices], 
                       columns=['treated_index', 'untreated_index', 'outcome_treated', 'outcome_untreated'])

att_many_to_one = (matched['outcome_treated'] - matched['outcome_untreated']).mean()
print("ATT (Many-to-One Matching):", att_many_to_one)

ATT (Many-to-One Matching): 3.8242995782569125


Stratification on the Propensity Score

In [11]:
df['stratum'] = pd.qcut(df['propensity_score'], q=5, labels=False)

# Initialize lists to store the weighted mean outcomes for treated and untreated groups within each stratum
weighted_mean_treated_list = []
weighted_mean_untreated_list = []

# Loop over each stratum to calculate the treatment effect
for stratum in df['stratum'].unique():
    stratum_data = df[df['stratum'] == stratum]
    treated = stratum_data[stratum_data['treatment'] == 1]
    untreated = stratum_data[stratum_data['treatment'] == 0]
    
    # Calculate the weighted mean outcomes for treated and untreated subjects
    treated_mean = treated['outcome'].mean()
    untreated_mean = untreated['outcome'].mean()
    
    # Store the results
    weighted_mean_treated_list.append(treated_mean)
    weighted_mean_untreated_list.append(untreated_mean)

# Calculate the overall treatment effect by averaging the treatment effects across strata
overall_treated_mean = np.mean(weighted_mean_treated_list)
overall_untreated_mean = np.mean(weighted_mean_untreated_list)
overall_att = overall_treated_mean - overall_untreated_mean

print(f'Estimated ATT using Stratification on the Propensity Score: {overall_att}')

Estimated ATT using Stratification on the Propensity Score: 4.043215998403173


Inverse Probability of Treatment Weighting Using the Propensity Score

In [12]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.3, random_state=1)

In [13]:
# Define the features (X), treatment (T), and outcome (Y) for training sets
X_train = train_data.drop(['treatment', 'outcome'], axis=1)
T_train = train_data['treatment']
Y_train = train_data['outcome']

In [14]:
# Estimate propensity scores for the training data
ps_model = LogisticRegression()
ps_model.fit(X_train, T_train)
propensity_scores_train = ps_model.predict_proba(X_train)[:, 1]

In [15]:
# Calculate IPTW weights for the training data
weights_train = T_train / propensity_scores_train + (1 - T_train) / (1 - propensity_scores_train)

In [16]:
# Calculate the weighted mean outcome for treated and untreated subjects in the training set
treated_weights_train = weights_train[T_train == 1]
untreated_weights_train = weights_train[T_train == 0]

treated_outcomes_train = Y_train[T_train == 1]
untreated_outcomes_train = Y_train[T_train == 0]

weighted_mean_treated = np.sum(treated_weights_train * treated_outcomes_train) / np.sum(treated_weights_train)
weighted_mean_untreated = np.sum(untreated_weights_train * untreated_outcomes_train) / np.sum(untreated_weights_train)

In [17]:
# Estimate the Average Treatment Effect (ATE) on training set
ate_iptw = weighted_mean_treated - weighted_mean_untreated
print(f'Estimated ATE using IPTW: {ate_iptw}')

Estimated ATE using IPTW: 3.951475213961511


Validate the results on the test set

In [18]:
X_test = test_data.drop(['treatment', 'outcome'], axis=1)
T_test = test_data['treatment']
Y_test = test_data['outcome']

propensity_scores_test = ps_model.predict_proba(X_test)[:, 1]

weights_test = T_test / propensity_scores_test + (1 - T_test) / (1 - propensity_scores_test)

treated_weights_test = weights_test[T_test == 1]
untreated_weights_test = weights_test[T_test == 0]
treated_outcomes_test = Y_test[T_test == 1]
untreated_outcomes_test = Y_test[T_test == 0]
weighted_mean_treated = np.sum(treated_weights_test * treated_outcomes_test) / np.sum(treated_weights_test)
weighted_mean_untreated = np.sum(untreated_weights_test * untreated_outcomes_test) / np.sum(untreated_weights_test)

ate_iptw_validation = weighted_mean_treated - weighted_mean_untreated
print(f'Estimated ATE using IPTW: {ate_iptw_validation}')


Estimated ATE using IPTW: 4.036568484902114


Double Machine Learning

In [19]:
%%capture
import numpy as np
from doubleml import DoubleMLData, DoubleMLPLR
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBRegressor, XGBClassifier
from sklearn.neural_network import MLPRegressor, MLPClassifier

X = df.drop(['treatment', 'outcome'], axis=1)
T = df['treatment']
Y = df['outcome']

# Create DoubleMLData object
dml_data = DoubleMLData.from_arrays(X.values, Y.values, T.values)

# Define the DML model using different ML algorithms
# Random Forest
ml_g_rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=1)
ml_m_rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=1)

# Gradient Boosting (XGBoost)
ml_g_xgb = XGBRegressor(n_estimators=100, max_depth=10, random_state=1)
ml_m_xgb = XGBClassifier(n_estimators=100, max_depth=10, random_state=1)

# Neural Network
ml_g_nn = MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, random_state=1)
ml_m_nn = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=1)

# Initialize DML models
dml_plr_rf = DoubleMLPLR(dml_data, ml_g_rf, ml_m_rf, n_folds=5)
dml_plr_xgb = DoubleMLPLR(dml_data, ml_g_xgb, ml_m_xgb, n_folds=5)
dml_plr_nn = DoubleMLPLR(dml_data, ml_g_nn, ml_m_nn, n_folds=5)

# Fit the models and estimate treatment effects
dml_plr_rf.fit()
treatment_effect_rf = dml_plr_rf.coef

dml_plr_xgb.fit()
treatment_effect_xgb = dml_plr_xgb.coef

dml_plr_nn.fit()
treatment_effect_nn = dml_plr_nn.coef

In [20]:
print(f'Estimated ATE using DML with Random Forest: {treatment_effect_rf}')
print(f'Estimated ATE using DML with XGBoost: {treatment_effect_xgb}')
print(f'Estimated ATE using DML with Neural Network: {treatment_effect_nn}')

Estimated ATE using DML with Random Forest: [3.9056936]
Estimated ATE using DML with XGBoost: [3.39407987]
Estimated ATE using DML with Neural Network: [3.7263446]


Bootstrapping for calculating the mean, variance and confidence interval 

In [21]:
from sklearn.utils import resample


n_bootstraps = 100
bootstrap_estimates_dml = []
bootstrap_estimates_psm = []

for _ in range(n_bootstraps):
    # Resample the data with replacement
    X_resampled, Y_resampled, T_resampled = resample(X.values, Y.values, T.values, random_state=_)

    ps_model = LogisticRegression()
    ps_model.fit(X_resampled, T_resampled)
    propensity_scores = ps_model.predict_proba(X_resampled)[:, 1]
    
    # Perform matching
    treated = np.where(T_resampled == 1)[0]
    untreated = np.where(T_resampled == 0)[0]
    indices, _ = pairwise_distances_argmin_min(propensity_scores[treated].reshape(-1, 1), propensity_scores[untreated].reshape(-1, 1))
    
    matched_outcomes = Y_resampled[untreated][indices]
    att_psm = (Y_resampled[treated] - matched_outcomes).mean()
    bootstrap_estimates_psm.append(att_psm)

# Convert bootstrap estimates to numpy arrays
bootstrap_estimates_psm = np.array(bootstrap_estimates_psm)

mean_estimate_psm = np.mean(bootstrap_estimates_psm)
variance_estimate_psm = np.var(bootstrap_estimates_psm)
ci_lower_psm = np.percentile(bootstrap_estimates_psm, 2.5)
ci_upper_psm = np.percentile(bootstrap_estimates_psm, 97.5)

print(f'PSM Mean Estimate: {mean_estimate_psm}')
print(f'PSM Variance Estimate: {variance_estimate_psm}')
print(f'PSM 95% Confidence Interval: [{ci_lower_psm}, {ci_upper_psm}]')

PSM Mean Estimate: 3.844899603871069
PSM Variance Estimate: 0.0747272705402616
PSM 95% Confidence Interval: [3.2402100948466854, 4.384405948380567]


In [22]:
bootstrap_estimates_iptw = []

for _ in range(n_bootstraps):
    # Resample the data with replacement
    X_resampled, Y_resampled, T_resampled = resample(X.values, Y.values, T.values, random_state=_)

    # Estimate propensity scores using logistic regression
    ps_model = LogisticRegression()
    ps_model.fit(X_resampled, T_resampled)
    propensity_scores = ps_model.predict_proba(X_resampled)[:, 1]

    # Calculate IPTW weights
    weights = T_resampled / propensity_scores + (1 - T_resampled) / (1 - propensity_scores)

    # Calculate the weighted mean outcome for treated and untreated subjects
    treated_weights = weights[T_resampled == 1]
    untreated_weights = weights[T_resampled == 0]

    treated_outcomes = Y_resampled[T_resampled == 1]
    untreated_outcomes = Y_resampled[T_resampled == 0]

    weighted_mean_treated = np.sum(treated_weights * treated_outcomes) / np.sum(treated_weights)
    weighted_mean_untreated = np.sum(untreated_weights * untreated_outcomes) / np.sum(untreated_weights)

    # Estimate the Average Treatment Effect (ATE) using IPTW
    ate_iptw = weighted_mean_treated - weighted_mean_untreated
    bootstrap_estimates_iptw.append(ate_iptw)

# Convert bootstrap estimates to numpy arrays
bootstrap_estimates_iptw = np.array(bootstrap_estimates_iptw)

# Calculate mean, variance, and confidence interval
mean_estimate_iptw = np.mean(bootstrap_estimates_iptw)
variance_estimate_iptw = np.var(bootstrap_estimates_iptw)
ci_lower_iptw = np.percentile(bootstrap_estimates_iptw, 2.5)
ci_upper_iptw = np.percentile(bootstrap_estimates_iptw, 97.5)

print(f'IPTW Mean Estimate: {mean_estimate_iptw}')
print(f'IPTW Variance Estimate: {variance_estimate_iptw}')
print(f'IPTW 95% Confidence Interval: [{ci_lower_iptw}, {ci_upper_iptw}]')

IPTW Mean Estimate: 4.071648117180177
IPTW Variance Estimate: 0.008868347056688413
IPTW 95% Confidence Interval: [3.8983551182153953, 4.266792178094139]


In [23]:
%%capture
# Number of bootstrap samples
n_bootstraps = 100
bootstrap_estimates_rf = []
bootstrap_estimates_xgb = []
bootstrap_estimates_nn = []
bootstrap_estimates_psm = []

for _ in range(n_bootstraps):
    # Resample the data with replacement
    X_resampled, Y_resampled, T_resampled = resample(X.values, Y.values, T.values, random_state=_)
    
    # For DML with Random Forest
    dml_data_resampled = DoubleMLData.from_arrays(X_resampled, Y_resampled, T_resampled)
    dml_plr_resampled_rf = DoubleMLPLR(dml_data_resampled, ml_g_rf, ml_m_rf, n_folds=5)
    dml_plr_resampled_rf.fit()
    bootstrap_estimates_rf.append(dml_plr_resampled_rf.coef[0])
    
    # For DML with XGBoost
    dml_plr_resampled_xgb = DoubleMLPLR(dml_data_resampled, ml_g_xgb, ml_m_xgb, n_folds=5)
    dml_plr_resampled_xgb.fit()
    bootstrap_estimates_xgb.append(dml_plr_resampled_xgb.coef[0])
    
    # For DML with Neural Network
    dml_plr_resampled_nn = DoubleMLPLR(dml_data_resampled, ml_g_nn, ml_m_nn, n_folds=5)
    dml_plr_resampled_nn.fit()
    bootstrap_estimates_nn.append(dml_plr_resampled_nn.coef[0])
    

# Convert bootstrap estimates to numpy arrays
bootstrap_estimates_rf = np.array(bootstrap_estimates_rf)
bootstrap_estimates_xgb = np.array(bootstrap_estimates_xgb)
bootstrap_estimates_nn = np.array(bootstrap_estimates_nn)

# Calculate mean, variance, and confidence intervals for the bootstrap estimates
mean_estimate_rf = np.mean(bootstrap_estimates_rf)
variance_estimate_rf = np.var(bootstrap_estimates_rf)
ci_lower_rf = np.percentile(bootstrap_estimates_rf, 2.5)
ci_upper_rf = np.percentile(bootstrap_estimates_rf, 97.5)

mean_estimate_xgb = np.mean(bootstrap_estimates_xgb)
variance_estimate_xgb = np.var(bootstrap_estimates_xgb)
ci_lower_xgb = np.percentile(bootstrap_estimates_xgb, 2.5)
ci_upper_xgb = np.percentile(bootstrap_estimates_xgb, 97.5)

mean_estimate_nn = np.mean(bootstrap_estimates_nn)
variance_estimate_nn = np.var(bootstrap_estimates_nn)
ci_lower_nn = np.percentile(bootstrap_estimates_nn, 2.5)
ci_upper_nn = np.percentile(bootstrap_estimates_nn, 97.5)

In [24]:
print(f'DML with Random Forest Mean Estimate: {mean_estimate_rf}')
print(f'DML with Random Forest Variance Estimate: {variance_estimate_rf}')
print(f'DML with Random Forest 95% Confidence Interval: [{ci_lower_rf}, {ci_upper_rf}]')

print(f'DML with XGBoost Mean Estimate: {mean_estimate_xgb}')
print(f'DML with XGBoost Variance Estimate: {variance_estimate_xgb}')
print(f'DML with XGBoost 95% Confidence Interval: [{ci_lower_xgb}, {ci_upper_xgb}]')

print(f'DML with Neural Network Mean Estimate: {mean_estimate_nn}')
print(f'DML with Neural Network Variance Estimate: {variance_estimate_nn}')
print(f'DML with Neural Network 95% Confidence Interval: [{ci_lower_nn}, {ci_upper_nn}]')

DML with Random Forest Mean Estimate: 3.837215832131098
DML with Random Forest Variance Estimate: 0.025676843540238976
DML with Random Forest 95% Confidence Interval: [3.4733378324623034, 4.114306663711916]
DML with XGBoost Mean Estimate: 3.2843688829568713
DML with XGBoost Variance Estimate: 0.046564539263560654
DML with XGBoost 95% Confidence Interval: [2.8811752963307105, 3.6915358469906008]
DML with Neural Network Mean Estimate: 3.8536769038256233
DML with Neural Network Variance Estimate: 0.0426011899443031
DML with Neural Network 95% Confidence Interval: [3.5121579780709804, 4.272169991247697]


Results

In [25]:
results = pd.DataFrame({
    'Method': ['Propensity Score', 'DML with Random Forest', 'DML with XGBoost', 'DML with Neural Network'],
    'Mean Estimate': [mean_estimate_iptw, mean_estimate_rf, mean_estimate_xgb, mean_estimate_nn],
    'ATE': [ate_iptw, treatment_effect_rf, treatment_effect_xgb, treatment_effect_nn],
    'Variance Estimate': [variance_estimate_psm, variance_estimate_rf, variance_estimate_xgb, variance_estimate_nn],
    '95% Confidence Interval': [
        f'[{ci_lower_psm}, {ci_upper_psm}]',
        f'[{ci_lower_rf}, {ci_upper_rf}]',
        f'[{ci_lower_xgb}, {ci_upper_xgb}]',
        f'[{ci_lower_nn}, {ci_upper_nn}]'
    ]
})

from tabulate import tabulate

print(tabulate(results, headers='keys', tablefmt='pretty'))

+---+-------------------------+--------------------+------------------+----------------------+------------------------------------------+
|   |         Method          |   Mean Estimate    |       ATE        |  Variance Estimate   |         95% Confidence Interval          |
+---+-------------------------+--------------------+------------------+----------------------+------------------------------------------+
| 0 |    Propensity Score     | 4.071648117180177  | 4.12387027030997 |  0.0747272705402616  | [3.2402100948466854, 4.384405948380567]  |
| 1 | DML with Random Forest  | 3.837215832131098  |   [3.9056936]    | 0.025676843540238976 | [3.4733378324623034, 4.114306663711916]  |
| 2 |    DML with XGBoost     | 3.2843688829568713 |   [3.39407987]   | 0.046564539263560654 | [2.8811752963307105, 3.6915358469906008] |
| 3 | DML with Neural Network | 3.8536769038256233 |   [3.7263446]    |  0.0426011899443031  | [3.5121579780709804, 4.272169991247697]  |
+---+-------------------------+---